In [69]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 有效广告及其特征

In [70]:
import pandas as pd
import numpy as np
from pathlib import Path
import os, re
from functools import reduce, partial
from utils import or_func, add_datepart, split_vals

In [71]:
root = Path('../test_B/')
root
os.listdir(root)

PosixPath('../test_B')

['Btest_sample_new.dat']

In [72]:
test_columns=['样本id','广告id','创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账户id','投放时段','人群定向','出价']
test = pd.read_csv(root/'Btest_sample_new.dat', sep='\t', header=None, names=test_columns, low_memory=False, encoding='utf-8')
print(len(test_columns))
test.head()
test.info()

11


,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38596 entries, 0 to 38595
Data columns (total 11 columns):
样本id      38596 non-null int64
广告id      38596 non-null int64
创建时间      38596 non-null int64
素材尺寸      38596 non-null int64
广告行业id    38596 non-null int64
商品类型      38596 non-null int64
商品id      38596 non-null int64
广告账户id    38596 non-null int64
投放时段      38596 non-null object
人群定向      38596 non-null object
出价        38596 non-null int64
dtypes: int64(9), object(2)
memory usage: 3.2+ MB


### 多值特征的处理
- 离散为 bool 特征

In [73]:
t = set()
def f(x): 
    y = x.split('|')
    for i in y:
        x = i.split(':')
        t.add(x[0])  

In [76]:
_ = test['人群定向'].apply(f)

In [7]:
t

{'age',
 'all',
 'area',
 'behavior',
 'connectionType',
 'consuptionAbility',
 'device',
 'education',
 'gender',
 'status',
 'work'}

In [8]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
test['is_all_field'] = test['人群定向'] == 'all'
test['price'] = test['出价']
test['time'] = test['投放时段']
test.drop(columns=['投放时段', '出价'], inplace=True)
test['创建时间'] = pd.to_datetime(test['创建时间'], unit='s')
test['has_product_id'] = test['商品id'] != -1  # - 1 表示的无商品 id
fill_v = 100.0  # 由训练集获取
test['price'].fillna(fill_v, inplace=True)
test['has_price'] = test['price'].isna()
test = test.reindex(columns=test.columns.to_list() + cols)

### 人群定向的分解

In [77]:
func = partial(split_vals, cols=cols, field='人群定向')

In [10]:
test = test.apply(func, axis=1)
test.drop(columns='人群定向', inplace=True)

- -999 表示空值

In [11]:
test.fillna(-999, inplace=True)

In [12]:
test['device'].value_counts()

-999                               33381
-9                                  4452
1200003                              208
1200002,1200003                      200
1200001,1200002,1200003              192
1200002                              156
1200001,1200002,1200003,1200004        7
Name: device, dtype: int64

In [13]:
def f_device(x):
    if x == '-999' or x == '-9' or isinstance(x, int):
        return x
    else:
        return ','.join(map(str, map(lambda t: int(t) - 1200000, x.split(','))))

In [14]:
y = test['device'].apply(f_device)

In [83]:
y.value_counts()

-999       33381
-9          4452
3            208
2,3          200
1,2,3        192
2            156
1,2,3,4        7
Name: device, dtype: int64

In [84]:
test['device'] = test['device'].apply(f_device)

In [17]:
cols3 = ['connectionType','consuptionAbility','device','education','gender','status','work']
for i in cols3:
    print(i)
    test[i].value_counts()

connectionType


-999         31236
-9            4452
4,2           2070
3,4,2          509
1,3,4,2        192
5,3,4,2        112
5,3,2,1,4       15
1,2             10
Name: connectionType, dtype: int64

consuptionAbility


-999    33657
-9       4452
3         257
2         192
2,3        38
Name: consuptionAbility, dtype: int64

device


...

-999     29543
-9        4452
3         2322
2         2269
1,3,2       10
Name: gender, dtype: int64

status


-999                  32280
-9                     4452
13,9,5                  412
13,9                    388
13,9,15,6               233
6,9,15                  190
6,9,15,7                172
7,11,14,15,2,6           99
7,11,14,15,2,6,9,5       87
6                        65
7,11,14,15,2,6,9         44
13,5                     42
13                       33
7,15,14                  33
13,9,15,4                17
6,15                     17
9                        11
7,14,15,6,9              11
7,11,14,15,6,9           10
Name: status, dtype: int64

work


-999    34134
-9       4452
6          10
Name: work, dtype: int64

In [18]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,152978,507977,639943,605859,742582
创建时间,2016-11-29 02:39:19,2019-03-05 13:52:08,2018-06-21 07:22:37,2019-03-21 09:52:14,2019-02-28 04:52:00
素材尺寸,30,30,34,40,40
广告行业id,118,122,84,98,122
商品类型,3,13,13,13,13
商品id,-1,629,2525,3808,7017
广告账户id,23511,4211,15582,10907,15855
is_all_field,False,True,False,True,False
price,37,54,161,28,12


- 时间特征

In [19]:
add_datepart(test, '创建时间', drop=True, time=True)

In [20]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,152978,507977,639943,605859,742582
素材尺寸,30,30,34,40,40
广告行业id,118,122,84,98,122
商品类型,3,13,13,13,13
商品id,-1,629,2525,3808,7017
广告账户id,23511,4211,15582,10907,15855
is_all_field,False,True,False,True,False
price,37,54,161,28,12
time,"281474976710655,281474976710655,28147497671065...","70368744161280,70368744161280,70368744161280,7...","281474976694272,281474976694272,28147497669427...","1099243192320,1099243192320,1099243192320,1099...","274936627200,274936627200,274936627200,2749366..."


In [21]:
test.shape

(38596, 38)

In [22]:
test.columns

Index(['样本id', '广告id', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id',
       'is_all_field', 'price', 'time', 'has_product_id', 'has_price', 'age',
       'area', 'behavior', 'connectionType', 'consuptionAbility', 'device',
       'education', 'gender', 'status', 'work', 'crt_dateYear',
       'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay', 'crt_dateDayofweek',
       'crt_dateDayofyear', 'crt_dateIs_month_end', 'crt_dateIs_month_start',
       'crt_dateIs_quarter_end', 'crt_dateIs_quarter_start',
       'crt_dateIs_year_end', 'crt_dateIs_year_start', 'crt_dateHour',
       'crt_dateMinute', 'crt_dateSecond', 'crt_dateElapsed'],
      dtype='object')

### 后处理

In [23]:
v_cols = ['样本id', '广告id', '广告行业id', '广告账户id', '商品id', '商品类型', '素材尺寸', 'is_all_field', 'has_product_id', 'has_price', 'time', 'age', 'area', 'behavior',
          'connectionType', 'consuptionAbility', 'device', 'education', 'gender', 'status', 'work', 'price', 'crt_dateYear', 'crt_dateMonth', 
          'crt_dateWeek', 'crt_dateDay', 'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end', 'crt_dateIs_month_start',
          'crt_dateIs_quarter_end', 'crt_dateIs_quarter_start', 'crt_dateIs_year_end', 'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateElapsed']

In [24]:
test['price'] = test['price'].astype('float64')

In [25]:
def fx(x):
    if x == '-999.0' or x == '-999' or x == -999.0 or x == -999:
        return -999
    else:
        return x

In [26]:
for i in v_cols:
    test[i] = test[i].apply(fx)

In [27]:
test['consuptionAbility'] = test['consuptionAbility'].astype('object')

训练集信息
```
Data columns (total 36 columns):
ad_id                       210855 non-null int64
ad_type_id                  210855 non-null int64
ad_account_id               210855 non-null int64
item_id                     210855 non-null int64
item_type                   210855 non-null int64
size                        210855 non-null int64
is_all_field                210855 non-null bool
has_product_id              210855 non-null bool
has_price                   210855 non-null bool
time                        210855 non-null object
age                         210855 non-null object
area                        210855 non-null object
behavior                    210855 non-null object
connectionType              210855 non-null object
consuptionAbility           210855 non-null object
device                      210855 non-null object
education                   210855 non-null object
gender                      210855 non-null float64
status                      210855 non-null object
work                        210855 non-null object
price                       210855 non-null float64
crt_dateYear                210855 non-null int64
crt_dateMonth               210855 non-null int64
crt_dateWeek                210855 non-null int64
crt_dateDay                 210855 non-null int64
crt_dateDayofweek           210855 non-null int64
crt_dateDayofyear           210855 non-null int64
crt_dateIs_month_end        210855 non-null bool
crt_dateIs_month_start      210855 non-null bool
crt_dateIs_quarter_end      210855 non-null bool
crt_dateIs_quarter_start    210855 non-null bool
crt_dateIs_year_end         210855 non-null bool
crt_dateIs_year_start       210855 non-null bool
crt_dateHour                210855 non-null int64
crt_dateElapsed             210855 non-null int64
target                      210855 non-null float64
dtypes: bool(9), float64(3), int64(14), object(10)
```

In [28]:
test[v_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38596 entries, 0 to 38595
Data columns (total 36 columns):
样本id                        38596 non-null int64
广告id                        38596 non-null int64
广告行业id                      38596 non-null int64
广告账户id                      38596 non-null int64
商品id                        38596 non-null int64
商品类型                        38596 non-null int64
素材尺寸                        38596 non-null int64
is_all_field                38596 non-null bool
has_product_id              38596 non-null bool
has_price                   38596 non-null bool
time                        38596 non-null object
age                         38596 non-null object
area                        38596 non-null object
behavior                    38596 non-null object
connectionType              38596 non-null object
consuptionAbility           38596 non-null object
device                      38596 non-null object
education                   38596 non-null object
gend

In [29]:
cz = ['样本id', '广告id', '广告行业id', '广告账户id', '商品id', '商品类型', '素材尺寸']
ez = ['sample_id', 'ad_id', 'ad_type_id', 'ad_account_id', 'item_id', 'item_type', 'size']
v_cols = ['sample_id', 'ad_id', 'ad_type_id', 'ad_account_id', 'item_id', 'item_type', 'size', 'is_all_field', 'has_product_id', 'has_price', 'time', 'age', 'area', 'behavior',
          'connectionType', 'consuptionAbility', 'device', 'education', 'gender', 'status', 'work', 'price', 'crt_dateYear', 'crt_dateMonth', 
          'crt_dateWeek', 'crt_dateDay', 'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end', 'crt_dateIs_month_start',
          'crt_dateIs_quarter_end', 'crt_dateIs_quarter_start', 'crt_dateIs_year_end', 'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateElapsed']

In [30]:
test['price'] = test['price'].apply(lambda x: np.log(1 + x))
for i, j in zip(cz, ez):
    test[j] = test[i]
test = test[v_cols]

In [31]:
test.columns
test.shape

Index(['sample_id', 'ad_id', 'ad_type_id', 'ad_account_id', 'item_id',
       'item_type', 'size', 'is_all_field', 'has_product_id', 'has_price',
       'time', 'age', 'area', 'behavior', 'connectionType',
       'consuptionAbility', 'device', 'education', 'gender', 'status', 'work',
       'price', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay',
       'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end',
       'crt_dateIs_month_start', 'crt_dateIs_quarter_end',
       'crt_dateIs_quarter_start', 'crt_dateIs_year_end',
       'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateElapsed'],
      dtype='object')

(38596, 36)

In [32]:
test.describe()

,sample_id,ad_id,ad_type_id,ad_account_id,item_id,item_type,size,price,crt_dateYear,crt_dateMonth,crt_dateWeek,crt_dateDay,crt_dateDayofweek,crt_dateDayofyear,crt_dateHour,crt_dateElapsed
count,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,38596.00000,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,38596.000000,3.859600e+04
mean,19298.500000,547482.729039,122.093015,14831.188517,8780.090502,7.43090,32.731604,3.958247,2018.699813,4.122448,16.208260,15.938413,2.428853,109.657452,7.780573,1.546251e+09
std,11141.849831,242031.364797,66.976285,8583.649131,10647.385704,6.06909,17.773940,1.165333,0.593527,2.965612,12.944879,7.391715,1.600020,91.486444,5.348986,1.409798e+07
min,1.000000,415.000000,-1.000000,80.000000,-1.000000,1.00000,1.000000,0.693147,2015.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.428400e+09
25%,9649.750000,347000.000000,68.000000,6930.000000,-1.000000,1.00000,30.000000,3.258097,2019.000000,3.000000,10.000000,11.000000,1.000000,65.000000,4.000000,1.546810e+09
50%,19298.500000,718761.000000,122.000000,15084.000000,2330.000000,12.00000,33.000000,4.025352,2019.000000,3.000000,12.000000,18.000000,2.000000,77.000000,7.000000,1.551919e+09
75%,28947.250000,740892.000000,172.000000,22373.000000,18392.750000,13.00000,40.000000,4.787492,2019.000000,3.000000,12.000000,21.000000,4.000000,81.000000,10.000000,1.552896e+09
max,38596.000000,746103.000000,244.000000,29742.000000,32864.000000,18.00000,66.000000,8.673684,2019.000000,12.000000,52.000000,31.000000,6.000000,365.000000,23.000000,1.553418e+09


In [33]:
test.head().T

,0,1,2,3,4
sample_id,1,2,3,4,5
ad_id,152978,507977,639943,605859,742582
ad_type_id,118,122,84,98,122
ad_account_id,23511,4211,15582,10907,15855
item_id,-1,629,2525,3808,7017
item_type,3,13,13,13,13
size,30,30,34,40,40
is_all_field,False,True,False,True,False
has_product_id,False,True,True,True,True
has_price,False,False,False,False,False


In [34]:
test.to_csv('../data/test_set_B.csv', index=None, encoding='utf-8')

- fin 测试集初步处理

In [104]:
test['price'].mode()

0    3.433987
dtype: float64